# Amber Prep Server - General Structure Test

This notebook tests the generalized `parse_structure` tool that can process:
- PDB database files (mmCIF or PDB format)
- Boltz-2/AlphaFold prediction outputs
- Any standard structure file

## Test Case: 1AKE (Adenylate Kinase)

- **PDB ID**: 1AKE
- **Chains**: A and B (homodimer)
- **Ligand**: AP5 (Bis(adenosine)-5'-pentaphosphate)
- **Goal**: Extract chain A + its bound AP5 ligand, prepare for MD


In [ ]:
# Setup path
import sys
sys.path.insert(0, '..')

from pathlib import Path
import json

# Test files
test_cif = Path("ak/1AKE.cif")
test_pdb = Path("ak/1AKE.pdb")

print(f"Test files:")
print(f"  mmCIF: {test_cif} - exists: {test_cif.exists()}")
print(f"  PDB:   {test_pdb} - exists: {test_pdb.exists()}")


In [ ]:
# Verify dependencies
print("Checking dependencies...")

try:
    import gemmi
    print(f"✓ gemmi available")
except ImportError:
    print("✗ gemmi not installed: pip install gemmi")

try:
    from rdkit import Chem
    print(f"✓ RDKit available")
except ImportError:
    print("✗ RDKit not installed (install via conda)")

try:
    import dimorphite_dl
    print(f"✓ dimorphite-dl available (pH-dependent protonation)")
except ImportError:
    print("⚠ dimorphite-dl not installed: pip install dimorphite-dl")
    print("  (will fall back to simple charge estimation)")

try:
    from common.base import BaseToolWrapper
    print(f"✓ common.base available")
except ImportError:
    print("✗ common.base not found")

# Check AmberTools
print("\nChecking AmberTools...")
from common.base import BaseToolWrapper

for tool_name in ["antechamber", "parmchk2", "pdb4amber", "tleap", "obabel"]:
    wrapper = BaseToolWrapper(tool_name, conda_env="mcp-md")
    status = "✓" if wrapper.is_available() else "✗"
    print(f"  {status} {tool_name}")


## Test 1: Parse Structure (mmCIF format)

Test the new `parse_structure` function with chain selection.


In [ ]:
# Import and reload the server module
import importlib
import servers.amber_prep_server as amber_module
importlib.reload(amber_module)

# Helper to get callable from FunctionTool
def get_callable(tool):
    """Extract callable from FunctionTool or return as-is"""
    if callable(tool):
        return tool
    for attr in ['fn', 'func', '_func', 'function', '_fn']:
        if hasattr(tool, attr):
            fn = getattr(tool, attr)
            if callable(fn):
                return fn
    raise TypeError(f"Cannot extract callable from {type(tool)}")

# Get parse_structure function
parse_structure = get_callable(amber_module.parse_structure)

print("Testing parse_structure with mmCIF format")
print("="*60)
print(f"Input: {test_cif}")
print(f"Select chains: ['A']")
print(f"Include ligands: True")
print(f"Exclude waters: True")
print()

# Parse structure - select chain A only
result = parse_structure(
    str(test_cif),
    select_chains=["A"],
    include_ligands=True,
    exclude_waters=True,
    ligand_distance_cutoff=5.0
)

print(f"✓ Job ID: {result['job_id']}")
print(f"✓ Output directory: {result['output_dir']}")
print(f"✓ File format: {result['file_format']}")

print(f"\n--- All Chains in Structure ---")
for chain in result['all_chains']:
    chain_type = "Protein" if chain['is_protein'] else "Ligand/Other"
    print(f"  Chain {chain['name']}: {chain_type}, {chain['num_ligands']} ligands, {chain['num_waters']} waters")

print(f"\n--- Selected Chains ---")
print(f"  {result['selected_chains']}")

print(f"\n--- Output Files ---")
print(f"  Protein: {result['protein_pdb']}")
for lig in result['ligand_files']:
    print(f"  Ligand: {lig}")

print(f"\n--- Extracted Ligands ---")
for lig in result['extracted_ligands']:
    in_chain = "(in selected chain)" if lig.get('in_selected_chain') else f"(nearby: {lig.get('note', '')})"
    print(f"  {lig['name']} chain {lig['chain']} #{lig['seqid']} {in_chain}")


## Test 2: Parse Structure (PDB format)

Test with PDB format input - should produce same results.


In [ ]:
print("Testing parse_structure with PDB format")
print("="*60)
print(f"Input: {test_pdb}")
print()

# Parse PDB format
result_pdb = parse_structure(
    str(test_pdb),
    select_chains=["A"],
    include_ligands=True,
    exclude_waters=True
)

print(f"✓ Job ID: {result_pdb['job_id']}")
print(f"✓ File format: {result_pdb['file_format']}")
print(f"✓ Selected chains: {result_pdb['selected_chains']}")
print(f"✓ Ligands found: {len(result_pdb['ligand_files'])}")

# Compare with mmCIF result
print(f"\n--- Comparison with mmCIF ---")
print(f"  mmCIF ligands: {len(result['ligand_files'])}")
print(f"  PDB ligands:   {len(result_pdb['ligand_files'])}")


## Test 3: Ligand Preparation with SMILES Template (Recommended)

Prepare the extracted ligand (AP5) using SMILES template matching.
This is the **recommended workflow** that ensures correct bond orders.


In [ ]:
# Get tool functions (using new SMILES template workflow with Dimorphite-DL protonation)
prepare_ligand_for_amber = get_callable(amber_module.prepare_ligand_for_amber)

# Optional: Manual SMILES for complex molecules (if CCD lookup fails)
# AP5 (Bis(adenosine)-5'-pentaphosphate)
MANUAL_SMILES = {
    # Use known SMILES dictionary as fallback
    "AP5": None,  # Will be fetched from CCD API
    "ATP": None,  
    "ADP": None,
}

# Optional: Manual charge override for complex molecules
# Use when automated charge calculation is unreliable
MANUAL_CHARGES = {
    # "AP5": -5,  # Example: manually specify charge for AP5
}

print("Preparing ligand for parameterization (SMILES Template + Dimorphite-DL)")
print("="*60)
print("Workflow:")
print("  1. Fetch SMILES from PDB CCD API (source of truth)")
print("  2. Apply pH 7.4 protonation using Dimorphite-DL")
print("  3. Assign bond orders from protonated SMILES template")
print("  4. Add hydrogens with correct geometry")
print("  5. Optimize structure with MMFF94")
print("  6. Calculate net charge from protonated molecule")
print()

job_dir = Path(result['output_dir'])
ligand_results = []

for lig_file in result['ligand_files']:
    lig_path = Path(lig_file)
    # Extract residue name from filename (ligand_AP5_chainA.pdb -> AP5)
    parts = lig_path.stem.split('_')
    res_name = parts[1] if len(parts) > 1 else "UNK"
    
    print(f"\nProcessing: {lig_path.name} (residue: {res_name})")
    print("-"*40)
    
    lig_result = {
        "file": str(lig_path),
        "residue": res_name,
        "charge": None,
        "smiles_source": None,
        "sdf_file": None
    }
    
    try:
        # Use SMILES template matching with Dimorphite-DL (Best Practice)
        print("[Step 1] Preparing ligand with SMILES template + pH protonation...")
        
        # Get manual SMILES/charge if available
        manual_smiles = MANUAL_SMILES.get(res_name)
        manual_charge = MANUAL_CHARGES.get(res_name)
        
        prep_result = prepare_ligand_for_amber(
            ligand_pdb=str(lig_path),
            ligand_id=res_name,
            smiles=manual_smiles,  # None = fetch from CCD
            output_dir=str(job_dir),
            optimize=True,  # Run MMFF94 optimization
            fetch_smiles=True,  # Try CCD API first
            target_ph=7.4,  # Physiological pH for protonation
            manual_charge=manual_charge  # Override charge if needed
        )
        
        print(f"  ✓ SDF output: {Path(prep_result['sdf_file']).name}")
        print(f"  ✓ SMILES source: {prep_result['smiles_source']}")
        print(f"  ✓ Target pH: {prep_result.get('target_ph', 'N/A')}")
        print(f"  ✓ Net charge: {prep_result['net_charge']} (from {prep_result.get('charge_source', 'unknown')})")
        if prep_result.get('mol_formal_charge') is not None:
            print(f"    - Mol formal charge: {prep_result['mol_formal_charge']}")
        print(f"  ✓ Atoms: {prep_result['num_atoms']} ({prep_result['num_heavy_atoms']} heavy)")
        print(f"  ✓ Optimized: {prep_result['optimized']}")
        if prep_result['optimized']:
            print(f"  ✓ Optimization converged: {prep_result['optimization_converged']}")
        
        # Show SMILES transformation (original → protonated)
        if prep_result.get('smiles_original') and prep_result.get('smiles_used'):
            print(f"\n  SMILES transformation:")
            print(f"    Original:   {prep_result['smiles_original'][:60]}...")
            print(f"    Protonated: {prep_result['smiles_used'][:60]}...")
        
        lig_result["sdf_file"] = prep_result['sdf_file']
        lig_result["charge"] = prep_result['net_charge']
        lig_result["smiles_source"] = prep_result['smiles_source']
        lig_result["charge_source"] = prep_result.get('charge_source', 'unknown')
        
        ligand_results.append(lig_result)
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        
        # Fallback: Try with manual SMILES/charge
        print("\n  → If failed, try providing SMILES in MANUAL_SMILES")
        print("  → Or specify charge in MANUAL_CHARGES")

# Summary
print(f"\n{'='*60}")
print("LIGAND PREPARATION SUMMARY (SMILES Template + Dimorphite-DL)")
print(f"{'='*60}")
for lr in ligand_results:
    status = "✓" if lr["sdf_file"] else "✗"
    source = f"(SMILES from {lr['smiles_source']}, charge from {lr.get('charge_source', 'unknown')})"
    print(f"  {status} {lr['residue']}: charge={lr['charge']} {source}")


## Test 4: Antechamber Force Field Generation (with SDF Input)

Generate GAFF2 parameters for the ligand using antechamber.
Now uses **SDF input** with correct bond orders from SMILES template.


In [ ]:
# Get antechamber function
run_antechamber_robust = get_callable(amber_module.run_antechamber_robust)
validate_frcmod = get_callable(amber_module.validate_frcmod)

print("Antechamber Force Field Generation (SDF Input)")
print("="*60)

antechamber_results = []

for lr in ligand_results:
    res_name = lr['residue']
    sdf_file = lr.get('sdf_file')  # Use SDF from prepare_ligand_for_amber
    charge = lr.get('charge')
    
    print(f"\nProcessing: {res_name}")
    print("-"*40)
    
    if sdf_file is None:
        print("  ✗ No SDF file available - run Test 3 first")
        continue
    
    if charge is None:
        print("  ✗ No charge calculated - run Test 3 first")
        continue
    
    print(f"  Input: {Path(sdf_file).name} (SDF with correct bond orders)")
    print(f"  Net charge: {charge}")
    
    try:
        print("  Running antechamber (this may take minutes to hours)...")
        ac_result = run_antechamber_robust(
            ligand_file=sdf_file,  # Use SDF file
            net_charge=charge,
            residue_name=res_name[:3].upper(),
            charge_method="bcc",
            atom_type="gaff2"
        )
        
        print(f"  ✓ MOL2: {Path(ac_result['mol2']).name}")
        print(f"  ✓ FRCMOD: {Path(ac_result['frcmod']).name}")
        print(f"  ✓ Charge used: {ac_result['charge_used']}")
        
        # Validate frcmod
        print("  Validating frcmod...")
        frcmod_result = validate_frcmod(ac_result['frcmod'])
        
        if frcmod_result['valid']:
            print("  ✓ frcmod validation PASSED")
        else:
            print(f"  ⚠ frcmod warnings: {frcmod_result['attn_count']} ATTN items")
        
        antechamber_results.append({
            "residue": res_name,
            "mol2": ac_result['mol2'],
            "frcmod": ac_result['frcmod'],
            "charge": ac_result['charge_used'],
            "frcmod_valid": frcmod_result['valid'],
            "success": True
        })
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()
        antechamber_results.append({
            "residue": res_name,
            "success": False,
            "error": str(e)
        })

# Summary
print(f"\n{'='*60}")
print("ANTECHAMBER SUMMARY")
print(f"{'='*60}")
for ar in antechamber_results:
    if ar.get('success'):
        print(f"  ✓ {ar['residue']}: charge={ar['charge']}")
    else:
        print(f"  ✗ {ar['residue']}: FAILED")


## Test 5: tleap System Building

Build the complete MD system with protein + ligand + solvent + ions.


In [ ]:
# Reload module and get functions
amber_module = importlib.reload(amber_module)
build_multi_ligand_system = get_callable(amber_module.build_multi_ligand_system)
prepare_protein_for_amber = get_callable(amber_module.prepare_protein_for_amber)

print("tleap System Building")
print("="*60)

# Get successful ligand parameterizations
successful_ligands = []
for ar in antechamber_results:
    if ar.get('success') and ar.get('mol2'):
        mol2_name = Path(ar['mol2']).stem
        if mol2_name.endswith('.gaff'):
            mol2_name = mol2_name[:-5]
        parts = mol2_name.split('_')
        resname = parts[1][:3].upper() if len(parts) >= 2 and parts[0] == 'ligand' else ar['residue'][:3].upper()
        successful_ligands.append({
            'mol2': ar['mol2'],
            'frcmod': ar['frcmod'],
            'residue_name': resname
        })

protein_pdb = result.get('protein_pdb')

if not protein_pdb:
    print("✗ No protein PDB available")
elif not successful_ligands:
    print("✗ No successful ligand parameterization")
else:
    print(f"Protein: {Path(protein_pdb).name}")
    print(f"Ligands: {len(successful_ligands)}")
    for i, lig in enumerate(successful_ligands):
        print(f"  [{i+1}] {lig['residue_name']}: {Path(lig['mol2']).name}")
    
    try:
        # Step 1: Prepare protein
        print(f"\n[Step 1] Preparing protein with pdb4amber...")
        protein_result = prepare_protein_for_amber(protein_pdb, output_dir=str(job_dir))
        print(f"  ✓ Prepared: {Path(protein_result['output_pdb']).name}")
        
        # Step 2: Build system
        print(f"\n[Step 2] Building system with tleap...")
        print(f"  Water model: TIP3P, Box padding: 10.0 Å, Box type: cubic")
        
        system_result = build_multi_ligand_system(
            protein_pdb=protein_result['output_pdb'],
            ligands=successful_ligands,
            output_dir=str(job_dir),
            forcefield="leaprc.protein.ff14SB",
            water_model="tip3p",
            box_padding=10.0,
            box_type="box",
            neutralize=True,
            salt_conc=0.0
        )
        
        print(f"\n  ✓ System built!")
        print(f"  ✓ Topology: {Path(system_result['parm7']).name}")
        print(f"  ✓ Coordinates: {Path(system_result['rst7']).name}")
        if system_result.get('num_atoms'):
            print(f"  ✓ Total atoms: {system_result['num_atoms']}")
        
        # Verify files
        parm7_path = Path(system_result['parm7'])
        rst7_path = Path(system_result['rst7'])
        if parm7_path.exists():
            print(f"  ✓ {parm7_path.name}: {parm7_path.stat().st_size/1024:.1f} KB")
        if rst7_path.exists():
            print(f"  ✓ {rst7_path.name}: {rst7_path.stat().st_size/1024:.1f} KB")
        
        print(f"\n{'='*60}")
        print("SYSTEM BUILDING COMPLETE - Ready for MD simulation!")
        print(f"{'='*60}")
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()


## Summary

This notebook demonstrated the generalized `parse_structure` tool:

1. **mmCIF and PDB support**: Both formats work identically
2. **Chain selection**: Extract specific chains (e.g., chain A only)
3. **Ligand detection**: Automatically includes ligands bound to selected chains
4. **Water exclusion**: Removes crystallographic waters
5. **Full MD preparation pipeline**: From structure file to MD-ready system

### Key differences from `parse_boltz2_complex`:

| Feature | `parse_boltz2_complex` | `parse_structure` |
|---------|----------------------|-------------------|
| Input format | mmCIF only | mmCIF + PDB |
| Chain selection | All chains | Selective |
| Ligand detection | Entity-based | Distance-based |
| Water handling | Not explicit | Configurable |
| Use case | Boltz-2 outputs | General purpose |

### Usage Example:

```python
from servers.amber_prep_server import parse_structure

# Extract chain A and its ligands from PDB structure
result = parse_structure(
    "1AKE.cif",           # or "1AKE.pdb"
    select_chains=["A"],  # Extract only chain A
    include_ligands=True, # Include bound ligands
    exclude_waters=True,  # Remove crystal waters
    ligand_distance_cutoff=5.0  # 5Å cutoff
)
```


## Test 6: 3D Visualization with py3Dmol

Visualize the built complex with py3Dmol (protein + ligand + solvent + ions).


In [ ]:
# Visualize tleap build result: Convert parm7/rst7 to PDB and display
import tempfile

try:
    import mdtraj as md
except ImportError:
    print("Installing MDTraj...")
    %pip install mdtraj
    import mdtraj as md

try:
    import py3Dmol
except ImportError:
    print("Installing py3Dmol...")
    %pip install py3Dmol
    import py3Dmol

# Check if we have tleap build results
if 'system_result' in dir() and system_result.get('parm7') and system_result.get('rst7'):
    parm7_path = Path(system_result['parm7'])
    rst7_path = Path(system_result['rst7'])
    
    print("=== tleap Build Validation ===")
    print(f"Topology: {parm7_path.name}")
    print(f"Coordinates: {rst7_path.name}")
    
    if parm7_path.exists() and rst7_path.exists():
        # Load coordinates with MDTraj
        print("\nLoading system with MDTraj...")
        traj = md.load(str(rst7_path), top=str(parm7_path))
        
        print(f"  Total atoms: {traj.n_atoms}")
        print(f"  Total residues: {traj.n_residues}")
        
        # Count residue types
        res_counts = {}
        ligand_resnames = set()
        standard_res = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
                        'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
                        'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'}
        water_res = {'WAT', 'HOH'}
        ion_res = {'NA', 'CL', 'Na+', 'Cl-', 'K', 'K+'}
        
        for res in traj.topology.residues:
            res_counts[res.name] = res_counts.get(res.name, 0) + 1
            if res.name not in standard_res and res.name not in water_res and res.name not in ion_res:
                ligand_resnames.add(res.name)
        
        # Print summary
        print(f"\n  Residue summary:")
        protein_count = sum(res_counts.get(aa, 0) for aa in standard_res)
        water_count = sum(res_counts.get(w, 0) for w in water_res)
        ion_count = sum(res_counts.get(i, 0) for i in ion_res)
        print(f"    Protein residues: {protein_count}")
        print(f"    Water molecules: {water_count}")
        print(f"    Ions: {ion_count}")
        print(f"    Ligands: {ligand_resnames}")
        for lig in ligand_resnames:
            print(f"      - {lig}: {res_counts.get(lig, 0)}")
        
        # Convert to PDB for visualization
        print("\nConverting to PDB for visualization...")
        with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
            tmp_pdb = tmp.name
        traj.save_pdb(tmp_pdb)
        
        with open(tmp_pdb, 'r') as f:
            pdb_content = f.read()
        Path(tmp_pdb).unlink()
        
        # Create viewer
        print("Creating 3D view...")
        view = py3Dmol.view(width=900, height=700)
        view.addModel(pdb_content, 'pdb')
        
        # Style protein - cartoon
        view.setStyle({'resn': list(standard_res)}, 
                      {'cartoon': {'color': 'spectrum'}})
        
        # Style ligands - stick with different colors
        lig_colors = ['green', 'cyan', 'magenta', 'orange']
        for i, resn in enumerate(sorted(ligand_resnames)):
            color = lig_colors[i % len(lig_colors)]
            view.setStyle({'resn': resn}, 
                          {'stick': {'color': color, 'radius': 0.3}})
            # Add label
            view.addResLabels({'resn': resn}, {
                'fontSize': 12,
                'fontColor': 'white',
                'backgroundColor': color,
                'backgroundOpacity': 0.8
            })
        
        # Style water - small dots (blue)
        view.setStyle({'resn': list(water_res)}, 
                      {'sphere': {'radius': 0.15, 'color': 'lightblue'}})
        
        # Style ions - spheres
        view.setStyle({'resn': ['NA', 'Na+']}, 
                      {'sphere': {'radius': 0.8, 'color': 'purple'}})
        view.setStyle({'resn': ['CL', 'Cl-']}, 
                      {'sphere': {'radius': 0.8, 'color': 'yellow'}})
        
        view.zoomTo()
        
        # Use orthographic projection
        view.setProjection('orthographic')
        
        # Add box visualization if available
        if traj.unitcell_lengths is not None:
            box = traj.unitcell_lengths[0] * 10  # nm to Angstrom
            print(f"\n  Box dimensions: {box[0]:.1f} x {box[1]:.1f} x {box[2]:.1f} Å")
        
        print(f"\n🔹 Protein ({protein_count} res): Cartoon (spectrum)")
        print(f"🔹 Ligands {list(ligand_resnames)}: Stick (colored)")
        print(f"🔹 Water ({water_count} mol): Dots (light blue)")
        print(f"🔹 Ions ({ion_count}): Spheres (Na+=purple, Cl-=yellow)")
        
        view.show()
    else:
        print(f"Files not found:")
        print(f"  parm7: {parm7_path.exists()}")
        print(f"  rst7: {rst7_path.exists()}")
else:
    print("No tleap build results available. Run Test 5 first.")


## Test 7: OpenMM Simulation (Minimize → Equilibrate → Production)

Run a minimal MD simulation with OpenMM to verify the system works.
- Platform: CUDA > OpenCL (Mac GPU) > CPU
- Ensemble: NPT (1 atm, 300 K)
- Short run for testing


In [ ]:
# OpenMM Simulation: Minimize → Equilibrate → Production
import time

try:
    import openmm as mm
    from openmm import app, unit
    from openmm.app import PDBFile, AmberPrmtopFile, AmberInpcrdFile
    from openmm.app import Simulation, StateDataReporter, DCDReporter
except ImportError:
    print("Installing OpenMM...")
    %pip install openmm
    import openmm as mm
    from openmm import app, unit
    from openmm.app import PDBFile, AmberPrmtopFile, AmberInpcrdFile
    from openmm.app import Simulation, StateDataReporter, DCDReporter

def select_platform():
    """Select best available platform: CUDA > OpenCL > CPU"""
    platform_preference = ['CUDA', 'OpenCL', 'CPU']
    
    print("Checking available platforms...")
    for name in platform_preference:
        try:
            platform = mm.Platform.getPlatformByName(name)
            # Test if platform actually works
            if name == 'CUDA':
                try:
                    platform.getPropertyDefaultValue('DeviceIndex')
                    print(f"  ✓ {name} available")
                    return platform, name
                except Exception:
                    print(f"  ✗ {name} not available (no GPU)")
                    continue
            elif name == 'OpenCL':
                print(f"  ✓ {name} available (Mac GPU)")
                return platform, name
            else:
                print(f"  ✓ {name} available")
                return platform, name
        except Exception as e:
            print(f"  ✗ {name} not available: {e}")
    
    raise RuntimeError("No suitable platform found!")

# Check if we have the topology/coordinate files
if 'system_result' in dir() and system_result.get('parm7') and system_result.get('rst7'):
    parm7_path = system_result['parm7']
    rst7_path = system_result['rst7']
    output_dir = Path(system_result['output_dir'])
    
    print("=" * 60)
    print("OpenMM MD Simulation")
    print("=" * 60)
    print(f"Topology: {Path(parm7_path).name}")
    print(f"Coordinates: {Path(rst7_path).name}")
    
    # Select platform
    platform, platform_name = select_platform()
    print(f"\n→ Using platform: {platform_name}")
    
    # Simulation parameters
    temperature = 300 * unit.kelvin
    pressure = 1 * unit.atmosphere
    timestep = 2 * unit.femtoseconds
    friction = 1 / unit.picosecond
    
    # Short runs for testing
    minimize_max_iter = 500
    equil_steps = 2500      # 5 ps equilibration
    prod_steps = 5000       # 10 ps production
    report_interval = 500   # Report every 1 ps
    
    print(f"\nSimulation parameters:")
    print(f"  Temperature: {temperature}")
    print(f"  Pressure: {pressure}")
    print(f"  Timestep: {timestep}")
    print(f"  Equilibration: {equil_steps} steps ({equil_steps * 2 / 1000} ps)")
    print(f"  Production: {prod_steps} steps ({prod_steps * 2 / 1000} ps)")
    
    # Load system
    print(f"\n[Step 1] Loading system...")
    t0 = time.time()
    prmtop = AmberPrmtopFile(parm7_path)
    inpcrd = AmberInpcrdFile(rst7_path)
    print(f"  ✓ Loaded in {time.time() - t0:.1f}s")
    print(f"  Atoms: {prmtop.topology.getNumAtoms()}")
    
    # Create system
    print(f"\n[Step 2] Creating OpenMM system...")
    t0 = time.time()
    system = prmtop.createSystem(
        nonbondedMethod=app.PME,
        nonbondedCutoff=10 * unit.angstrom,
        constraints=app.HBonds,
        rigidWater=True
    )
    
    # Add barostat for NPT
    system.addForce(mm.MonteCarloBarostat(pressure, temperature, 25))
    print(f"  ✓ System created in {time.time() - t0:.1f}s")
    
    # Create integrator and simulation
    integrator = mm.LangevinMiddleIntegrator(temperature, friction, timestep)
    simulation = Simulation(prmtop.topology, system, integrator, platform)
    simulation.context.setPositions(inpcrd.positions)
    
    if inpcrd.boxVectors is not None:
        simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
    
    # Energy minimization
    print(f"\n[Step 3] Energy minimization (max {minimize_max_iter} steps)...")
    t0 = time.time()
    state_before = simulation.context.getState(getEnergy=True)
    energy_before = state_before.getPotentialEnergy().value_in_unit(unit.kilojoules_per_mole)
    
    simulation.minimizeEnergy(maxIterations=minimize_max_iter)
    
    state_after = simulation.context.getState(getEnergy=True)
    energy_after = state_after.getPotentialEnergy().value_in_unit(unit.kilojoules_per_mole)
    print(f"  ✓ Minimized in {time.time() - t0:.1f}s")
    print(f"  Energy: {energy_before:.1f} → {energy_after:.1f} kJ/mol")
    
    # Initialize velocities
    simulation.context.setVelocitiesToTemperature(temperature)
    
    # Setup reporters
    dcd_file = output_dir / "trajectory.dcd"
    log_file = output_dir / "simulation.log"
    
    simulation.reporters.append(DCDReporter(str(dcd_file), report_interval))
    simulation.reporters.append(StateDataReporter(
        str(log_file), report_interval,
        step=True, time=True, potentialEnergy=True, kineticEnergy=True,
        totalEnergy=True, temperature=True, volume=True, density=True,
        speed=True
    ))
    simulation.reporters.append(StateDataReporter(
        sys.stdout, report_interval,
        step=True, time=True, temperature=True, speed=True, remainingTime=True,
        totalSteps=equil_steps + prod_steps
    ))
    
    # Equilibration (NVT heating is implicit, we go straight to NPT)
    print(f"\n[Step 4] NPT Equilibration ({equil_steps * 2 / 1000} ps)...")
    t0 = time.time()
    simulation.step(equil_steps)
    print(f"  ✓ Equilibration done in {time.time() - t0:.1f}s")
    
    # Production
    print(f"\n[Step 5] Production ({prod_steps * 2 / 1000} ps)...")
    t0 = time.time()
    simulation.step(prod_steps)
    print(f"  ✓ Production done in {time.time() - t0:.1f}s")
    
    # Save final state
    final_pdb = output_dir / "final_state.pdb"
    state = simulation.context.getState(getPositions=True, getVelocities=True)
    with open(final_pdb, 'w') as f:
        PDBFile.writeFile(simulation.topology, state.getPositions(), f)
    print(f"\n✓ Final state saved: {final_pdb.name}")
    
    # Summary
    print(f"\n{'='*60}")
    print("SIMULATION COMPLETE")
    print(f"{'='*60}")
    print(f"  Output directory: {output_dir}")
    print(f"  Trajectory: {dcd_file.name}")
    print(f"  Log: {log_file.name}")
    print(f"  Final PDB: {final_pdb.name}")
    
else:
    print("No topology/coordinate files available. Run system building first.")


## Test 8: Trajectory Visualization with py3Dmol

Visualize the MD trajectory (equilibration + production) with py3Dmol animation.


In [ ]:
# Trajectory visualization with py3Dmol
import numpy as np
import tempfile

try:
    import mdtraj as md
except ImportError:
    print("Installing MDTraj...")
    %pip install mdtraj
    import mdtraj as md

import py3Dmol

# Check if we have trajectory files
if 'system_result' in dir() and system_result.get('output_dir'):
    output_dir = Path(system_result['output_dir'])
    dcd_file = output_dir / "trajectory.dcd"
    parm7_file = Path(system_result['parm7'])
    
    if dcd_file.exists() and parm7_file.exists():
        print("Loading trajectory...")
        
        # Load trajectory with MDTraj
        traj = md.load(str(dcd_file), top=str(parm7_file))
        print(f"  Frames: {traj.n_frames}")
        print(f"  Atoms: {traj.n_atoms}")
        print(f"  Time: {traj.time[0]:.1f} - {traj.time[-1]:.1f} ps")
        
        # Select protein and ligand atoms (exclude water and ions)
        protein_indices = traj.topology.select('protein')
        
        # Find ALL ligand residues (any non-standard non-water residue)
        lig_indices = []
        ligand_resnames = set()
        ligand_details = []  # For debugging
        standard_res = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
                        'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
                        'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL',
                        'WAT', 'HOH', 'NA', 'CL', 'Na+', 'Cl-'}
        for residue in traj.topology.residues:
            if residue.name not in standard_res:
                atom_indices = [atom.index for atom in residue.atoms]
                lig_indices.extend(atom_indices)
                ligand_resnames.add(residue.name)
                ligand_details.append(f"{residue.name}:{residue.resSeq} ({len(atom_indices)} atoms)")
        lig_indices = np.array(lig_indices) if lig_indices else np.array([], dtype=int)
        
        # Debug: show all found ligands
        print(f"  Found ligand residues:")
        for detail in ligand_details[:10]:  # Limit output
            print(f"    - {detail}")
        if len(ligand_details) > 10:
            print(f"    ... and {len(ligand_details) - 10} more")
        
        # Combine protein + ligand
        if len(lig_indices) > 0:
            keep_indices = np.concatenate([protein_indices, lig_indices])
        else:
            keep_indices = protein_indices
        
        # Remove duplicates and sort
        keep_indices = np.unique(keep_indices)
        
        # Ensure indices are within range
        keep_indices = keep_indices[keep_indices < traj.n_atoms]
        
        print(f"  Protein atoms: {len(protein_indices)}")
        print(f"  Ligand atoms: {len(lig_indices)}")
        print(f"  Ligand types: {ligand_resnames if ligand_resnames else 'None'}")
        print(f"  Selection atoms: {len(keep_indices)}")
        
        # Subset trajectory to protein + ligand only
        traj_subset = traj.atom_slice(keep_indices)
        print(f"  Visualization atoms: {traj_subset.n_atoms}")
        
        # Sample frames for visualization
        max_frames = 15
        if traj_subset.n_frames > max_frames:
            frame_indices = np.linspace(0, traj_subset.n_frames - 1, max_frames, dtype=int)
            traj_viz = traj_subset[frame_indices]
            print(f"  Sampled {max_frames} frames for visualization")
        else:
            traj_viz = traj_subset
        
        print("\nPreparing visualization...")
        
        # Save all frames to a single multi-model PDB file
        # This is the proper way to do trajectory animation in py3Dmol
        with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
            tmp_path = tmp.name
        
        # Write all frames as MODEL/ENDMDL blocks
        with open(tmp_path, 'w') as f:
            for frame_idx in range(traj_viz.n_frames):
                frame = traj_viz[frame_idx]
                # Save single frame to temp
                frame_tmp = tmp_path + f".frame{frame_idx}.pdb"
                frame.save_pdb(frame_tmp, force_overwrite=True)
                with open(frame_tmp, 'r') as ff:
                    content = ff.read()
                # Wrap in MODEL/ENDMDL
                f.write(f"MODEL     {frame_idx + 1}\n")
                # Remove any existing MODEL/ENDMDL lines
                for line in content.split('\n'):
                    if not line.startswith('MODEL') and not line.startswith('ENDMDL') and line.strip():
                        f.write(line + '\n')
                f.write("ENDMDL\n")
                Path(frame_tmp).unlink()
        
        # Read the multi-model PDB
        with open(tmp_path, 'r') as f:
            pdb_content = f.read()
        Path(tmp_path).unlink()
        
        # Create viewer
        view = py3Dmol.view(width=800, height=600)
        view.addModelsAsFrames(pdb_content, 'pdb')
        
        # Style - apply to all frames
        aa_list = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU', 
                   'GLY', 'HIS', 'HID', 'HIE', 'HIP', 'ILE', 'LEU', 'LYS', 
                   'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
        
        view.setStyle({'resn': aa_list}, {'cartoon': {'color': 'spectrum'}})
        
        # Style all ligands with different colors and add labels
        lig_colors = ['green', 'cyan', 'magenta', 'orange']
        for i, resn in enumerate(sorted(ligand_resnames)):
            color = lig_colors[i % len(lig_colors)]
            view.setStyle({'resn': resn}, {'stick': {'color': color, 'radius': 0.3}})
            # Add label for each ligand (at center of residue)
            view.addResLabels({'resn': resn}, {
                'fontSize': 12,
                'fontColor': 'white',
                'backgroundColor': color,
                'backgroundOpacity': 0.8
            })
        
        view.zoomTo()
        
        # Use orthographic projection
        view.setProjection('orthographic')
        
        # Enable frame-based animation
        view.animate({'loop': 'forward', 'reps': 0, 'interval': 100})
        
        print(f"\n🔹 Protein: Cartoon (spectrum)")
        print(f"🔹 Ligands: {list(ligand_resnames)} (stick, different colors)")
        print(f"🔹 Frames: {traj_viz.n_frames} (animated)")
        print(f"\n▶️ Animation should auto-play")
        print(f"   If static, try opening in browser or use nglview instead")
        
        view.show()
    else:
        print(f"Trajectory file not found: {dcd_file}")
else:
    print("No simulation results available. Run simulation first.")
